### Importing Libraries

This cell imports the necessary libraries for the notebook, including PyTorch, the Hugging Face transformers library, the datasets library for loading datasets, and the peft library for applying LoRA configurations.

In [1]:
import os 
from collections import Counter
import torch
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


### Setting Up Environment

This cell clears the CUDA cache to manage memory effectively and sets up the Weights and Biases (WandB) project for logging training metrics.

In [2]:
# Configure memory management first
torch.cuda.empty_cache()

# Define response template
RESPONSE_TEMPLATE = "<response>"  

## Loading and Preparing the Dataset

Now, we extracts unique sentiment labels from the dataset and calculates their distribution, which is useful for understanding the dataset's balance.

In [ ]:
# Get distinct classes from the dataset
def get_distinct_classes(dataset):
    # Extract all output labels from the training set
    if isinstance(dataset, DatasetDict) and "train" in dataset:
        labels = dataset["train"]["output"]
        
        # Count unique labels and their distribution
        label_counts = Counter(labels)
        
        # Get unique labels
        unique_labels = set(labels)
        
        # Convert to percentage distribution
        total_samples = len(labels)
        
        label_distribution = {label: count / total_samples * 100 for label, count in label_counts.items()}

        # Print results
        print(f"Unique labels count: {len(label_counts)}")
        print("Label distribution:", label_distribution)
        
        return unique_labels
    
    return set() 

This function loads the FinGPT sentiment analysis dataset, splits it into training and validation sets if necessary, and prints the dataset statistics.

In [3]:
# 1. Load and prepare the dataset
def load_and_prepare_dataset():
    print("Loading dataset...")

    # load FinGPT dataset
    dataset = load_dataset("FinGPT/fingpt-sentiment-train")
    print("Successfully loaded FinGPT/fingpt-sentiment-train dataset")
    
    # Split dataset into train and validation sets if only train split exists
    if isinstance(dataset, DatasetDict) and "train" in dataset and "validation" not in dataset:
        train_test_split = dataset["train"].train_test_split(test_size=0.1)
        dataset = DatasetDict({
            "train": train_test_split["train"],
            "validation": train_test_split["test"]
        })
    
    # Print dataset statistics
    print(f"Dataset loaded with {len(dataset['train'])} training samples and {len(dataset['validation'])} validation samples")
    
    # Get distinct classes
    distinct_classes = get_distinct_classes(dataset)
    
    return dataset, distinct_classes


## Loading the Model and Tokenizer

This function loads the Mistral 7B model and its tokenizer, configuring them for use in the training process. It also handles special tokens required for the model.

In [5]:
def load_model_and_tokenizer(model_name, cache_dir):
    print("Loading model and tokenizer...")
    os.makedirs(cache_dir, exist_ok=True)
    
    # Load tokenizer first with proper special tokens
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        # padding_side="right",
        eos_token="<|im_end|>",
        bos_token="<s>",
        pad_token="<pad>",
        additional_special_tokens=[RESPONSE_TEMPLATE,"<response|end>"]
    )
    print(f"Tokenizer loaded with vocab size: {len(tokenizer)}")
    
    # Load model with quantization and device configuration
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        device_map="auto",
        torch_dtype=torch.float16,
        attn_implementation="flash_attention_2"
    )
    print("Model loaded!")
    
    
    # Resize token embeddings to match tokenizer
    model.resize_token_embeddings(len(tokenizer))
    
    print("Model and tokenizer loaded successfully")

    for p in model.parameters():
        if not p.requires_grad:
            p.data = p.to(torch.float16)
    
    return model, tokenizer

## Define LoRA Configuration

This function sets up the LoRA configuration parameters, which dictate how the model will be fine-tuned.

In [6]:
def apply_lora_config():
    # Define LoRA configuration
    peft_config = LoraConfig(
        r=16,                   # Rank dimension
        lora_alpha=32,          # Alpha parameter for LoRA scaling
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj", 
            "up_proj",
            "down_proj"
        ],
        lora_dropout=0.1,      # dropout for regularization
        bias="none",            # Don't train bias terms
        task_type=TaskType.CAUSAL_LM  # Task type: causal language modeling
    )
    
    return peft_config


## Preprocess Data

This function formats the dataset for sentiment classification, creating input-output pairs that the model will use during training.

In [7]:
def preprocess_data(example):
    """Preprocess the data to format for sentiment classification."""
    # Format the prompt with sentiment label appended directly
    RESPONSE_END_TEMPLATE="<response|end>"
    prompt = (
        f"Instruction: {example['instruction']}\n"
        f"Input: {example['input']}\n"
        f"{RESPONSE_TEMPLATE}{example['output']}{RESPONSE_END_TEMPLATE}"
    )
    return {"text": prompt}


In [8]:
def print_parameters(model):
    """Print parameter statistics for the model."""
    # Count total parameters
    total_params = sum(p.numel() for p in model.parameters())

    # Count trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Compute percentage
    trainable_percentage = (trainable_params / total_params) * 100
    print(f"Trainable params: {trainable_params:,} || All params: {total_params:,} || Trainable%: {trainable_percentage:.4f}%")

In [9]:
# Check available CUDA devices
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Using CUDA with {device_count} available device(s)")
    for i in range(device_count):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
        print(f"Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
else:
    print("CUDA is not available. Using CPU.")

Using CUDA with 1 available device(s)
Device 0: NVIDIA RTX A6000
Memory: 51.03 GB


In [10]:
# 1. Load dataset
dataset, distinct_classes = load_and_prepare_dataset()

print(dataset)
print("Distinct classes:", distinct_classes)

Loading dataset...
Successfully loaded FinGPT/fingpt-sentiment-train dataset
Dataset loaded with 69094 training samples and 7678 validation samples
Unique labels count: 9
Label distribution: {'neutral': 38.078559643384374, 'strong positive': 0.2807769126118042, 'mildly positive': 3.311430804411382, 'moderately positive': 8.045561119634122, 'positive': 28.16597678524908, 'negative': 15.244449590413062, 'mildly negative': 2.7484296755145166, 'moderately negative': 3.844038556169856, 'strong negative': 0.2807769126118042}
DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 69094
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 7678
    })
})
Distinct classes: {'neutral', 'strong positive', 'strong negative', 'moderately positive', 'positive', 'moderately negative', 'negative', 'mildly negative', 'mildly positive'}


In [11]:
# 2. Load model and tokenizer
model_name = "mistralai/Mistral-7B-v0.1"
cache_dir = "hfcache"

model, tokenizer = load_model_and_tokenizer(model_name, cache_dir)

Loading model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Model loaded with quantization


The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Model and tokenizer loaded successfully


In [12]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32004, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [14]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<response>', '<response|end>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<response>", rstrip=False, lstr

In [15]:
# 3. Configure LoRA
lora_config = apply_lora_config()
lora_config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'up_proj', 'v_proj', 'o_proj', 'gate_proj', 'down_proj', 'q_proj', 'k_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [16]:
# 4. Preprocess the dataset
print("Preprocessing dataset...")

# Preprocess dataset
tokenized_dataset = dataset.map(
    preprocess_data,
    batched=False,
    remove_columns=["instruction", "input", "output"]
)

# Print sample for verification
print(tokenized_dataset)
print("Sample processed example:", tokenized_dataset["train"][0])

Preprocessing dataset...


Map:   0%|          | 0/69094 [00:00<?, ? examples/s]

Map:   0%|          | 0/7678 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 69094
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 7678
    })
})
Sample processed example: {'text': 'Instruction: What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}.\nInput: Yahoo Finance Live anchors Brian Sozzi, Brad Smith and Julie Hyman discuss second-quarter earnings for Amazon.\n<response>neutral<response|end>'}


In [17]:
# 5. Create data collator with completion-only loss
response_template_ids = tokenizer.encode(RESPONSE_TEMPLATE, add_special_tokens=False)
print(f"Response template token IDs: {response_template_ids}")

collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template=RESPONSE_TEMPLATE
)

Response template token IDs: [32002]


## Define Training Arguments

Now, we will set up the training arguments, including batch size, learning rate, and other hyperparameters that control the training process.

In [18]:
# 6. Configure training arguments
model_output_dir = "financial-sentiment-LoRA"
os.makedirs(model_output_dir, exist_ok=True)
    
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=8,         
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=500,                        
    save_strategy="no",                 
    num_train_epochs=1,
    learning_rate=5e-4,
    fp16=True,
    logging_steps=5,
    max_grad_norm=1.0,                 
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    report_to=["wandb"],
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    hub_private_repo=True,
    push_to_hub=True,                    
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Initialize the Trainer

Next, we initialize the SFTTrainer with the model, training arguments, datasets, and other configurations.

In [19]:
# 7. Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,  
    peft_config=lora_config,
    data_collator=collator
)

/tmp/ipykernel_2347/1475081074.py:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/69094 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/69094 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/69094 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/69094 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/7678 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/7678 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/7678 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/7678 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
print_parameters(trainer.model)

Trainable params: 41,943,040 || All params: 7,283,707,904 || Trainable%: 0.5758%


## Train the Model

In [21]:
# Train the model
print("Training the model...")
trainer.train()

# Save the model
trainer.save_model()
print("Model saved successfully!")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training the model...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ankitamungalpara13 (ankitamungalpara13-university-of-massachusetts-dartmouth) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
500,5.661800,5.774400
1000,3.472400,3.641669


...

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/692M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

Model saved successfully!


In [22]:
# Restart and run 
!nvidia-smi

Sun Mar 30 17:29:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               On  |   00000000:46:00.0 Off |                  Off |
| 31%   41C    P8             27W /  300W |   22724MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch

In [24]:

peft_model_id = "AnkitaMungalpara/financial-sentiment-sftmodel-LoRA"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
print(config.base_model_name_or_path)


adapter_config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

mistralai/Mistral-7B-v0.1


In [25]:
RESPONSE_TEMPLATE

'<response>'

In [26]:

# When loading the base model, make sure to add the special tokens
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(
        config.base_model_name_or_path,
        eos_token="<|im_end|>",
        bos_token="<s>",
        pad_token="<pad>",
        additional_special_tokens=[RESPONSE_TEMPLATE,"<response|end>"]
    )
# Resize the model's token embeddings to match the tokenizer
model.resize_token_embeddings(len(tokenizer))

# Now load the PEFT model
model = PeftModel.from_pretrained(model, peft_model_id)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/692M [00:00<?, ?B/s]

In [27]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32004, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [28]:

model.to(torch.float16)
model.cuda()
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32004, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [29]:
RESPONSE_TEMPLATE

'<response>'

In [30]:
# tokenizer.additional_special_tokens
response_end_id = tokenizer.convert_tokens_to_ids("<response|end>")
print(f"Token ID for <response|end>: {response_end_id}")


Token ID for <response|end>: 32003


## Inference: Classifying Sentiments

This function generates sentiment classification responses using the fine-tuned model.

In [31]:
import torch

def infer_sentiment(instruction, text, model, tokenizer, response_template=f"{RESPONSE_TEMPLATE}", max_new_tokens=4):
    """Generates sentiment classification response using fine-tuned LoRA model."""
    
    # Ensure the model is on CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    input_text = f"Instruction: {instruction}\nInput: {text}\n{response_template}"

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt").to(device)  # Move inputs to the same device as the model

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, 
                         max_new_tokens=max_new_tokens, 
                         do_sample=True, 
                         top_p=0.99, 
                         temperature=0.001, 
                         repetition_penalty=1.1, 
                         eos_token_id=tokenizer.convert_tokens_to_ids("<response|end>"))

    # Decode output text
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    #print(response)
    sentiment = response.split(response_template)[-1].strip()
    
    return sentiment


In [32]:
instruction = f'What is the sentiment of this news? Please choose an answer from strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive.'
text="Starbucks says the workers violated safety policies while workers said they'd never heard of the policy before and are alleging retaliation."

infer_sentiment(instruction, text, model, tokenizer, response_template="<response>")

Setting `pad_token_id` to `eos_token_id`:32003 for open-end generation.


'moderately positive'

In [33]:
instruction = "What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive"
text="Here we highlight some top-ranked technology ETFs that investors can consider betting on as a rebound rally is witnessed in tech stocks."
infer_sentiment(instruction, text, model, tokenizer, response_template="<response>")

Setting `pad_token_id` to `eos_token_id`:32003 for open-end generation.


'moderately positive'

In [34]:
instruction = "What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}."
text="Here we highlight some top-ranked technology ETFs that investors can consider betting on as a rebound rally is witnessed in tech stocks."
infer_sentiment(instruction, text, model, tokenizer, response_template="<response>")

Setting `pad_token_id` to `eos_token_id`:32003 for open-end generation.


'moderately positive'